<a href="https://colab.research.google.com/github/diegofrl/BT---Cross-Lingual-Classification/blob/main/POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install transformers
!pip install gtts
!pip install pydub
!touch empty.mp3
!git clone https://github.com/diegofrl/BT---Cross-Lingual-Classification.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel
import string
from tensorflow import keras
from keras.models import load_model
import pickle

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased', num_labels=4)
model = AutoModel.from_pretrained('bert-base-multilingual-uncased', num_labels=4)

def vectorize_text(text):
    def is_subword(token):
        return token[:2] == "##"

    text = text.lower()
    tokens = tokenizer.tokenize(text)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    with torch.no_grad():
        embeddings = model(torch.tensor([input_ids]))[0]

    word_vectors = []
    words = []
    current_word = ""
    current_word_vector = None

    for i in range(1, len(tokens)-1):
        token = tokens[i]
        if not is_subword(token):
            if current_word:
                words.append(current_word)
                word_vectors.append(current_word_vector)
            current_word = token
            current_word_vector = embeddings[0][i].numpy()
        else:
            current_word += token[2:]
            current_word_vector += embeddings[0][i].numpy()

    if current_word:
        words.append(current_word)
        word_vectors.append(current_word_vector)
    
    df = pd.DataFrame({'word': words, 'vector': word_vectors})
    return df


def tf_get_predictions(text, model):
    df = vectorize_text(text)
    word_label_tuples = []
    merged_tuples = []

    for word, vector in zip(df['word'], df['vector']):
        sample = np.array(vector).reshape(1, -1)
        y_pred_one_hot = model.predict(sample, verbose=0)
        label = np.argmax(y_pred_one_hot, axis=1)[0]
        labels_str = {0: 'de', 1:'en', 2: 'fr', 3: 'it'}
        label_str = labels_str.get(label)
        word_label_tuples.append([word, label_str])

    # Merge tuples of the same language
    for word_label in word_label_tuples:
        word, label = word_label
        if not merged_tuples:
            merged_tuples.append([word, label])
        else:
            last_tuple = merged_tuples[-1]
            if label == last_tuple[1]:
                last_tuple[0] += ' ' + word
            else:
                merged_tuples.append([word, label])

    return merged_tuples

# Load the neural network model
nn_model = load_model('/content/BT---Cross-Lingual-Classification/ML_Models/transformer_nn_model.h5')

text = "How do you say 'Ich arbeite in Zürich' in English?"
predictions = tf_get_predictions(text, nn_model)
print(predictions)


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[['how', 'en'], ['do', 'it'], ["you say '", 'en'], ['ich arbeite in zurich', 'de'], ["' in english ?", 'en']]


In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import tempfile

def predictions_to_html(predictions):
    html = ""
    for prediction in predictions:
      if prediction[1] == "en":
        html += "<span style='color: red'>"
      elif prediction[1] == "fr":
        html += "<span style='color: blue'>"
      elif prediction[1] == "de":
        html += "<span style='color: yellow'>"
      elif prediction[1] == "it":
        html += "<span style='color: green'>"
      else:
        html += "<span>"      
      html += prediction[0]+"</span> "
    return html

def crosslingual_tts(predictions, output_path):
    # Initialize an empty audio segment
    result_audio = AudioSegment.empty()

    # Loop through the divs, extract text and language, and generate speech using gTTS
    for prediction in predictions:
        text = prediction[0]
        lang = prediction[1]

        # Convert text to speech using gTTS
        tts = gTTS(text=text, lang=lang)
        with tempfile.NamedTemporaryFile(delete=True) as audio_buffer:
            tts.save(audio_buffer.name)

            # Load the audio segment and concatenate it to the result
            audio_segment = AudioSegment.from_file(audio_buffer.name, format='mp3')
            result_audio += audio_segment

    # Save the result as an mp3 file
    result_audio.export(output_path, format='mp3')


print(predictions_to_html(predictions))

<span style='color: red'>how</span> <span style='color: green'>do</span> <span style='color: red'>you say '</span> <span style='color: yellow'>ich arbeite in zurich</span> <span style='color: red'>' in english ?</span> 


In [ ]:
import gradio as gr

def gradio_call(input_text):
    if not input_text or input_text == "":
        return "<span style='color: red; font-weight: bold;'>ERROR: Please provide an input text</span>", "empty.mp3"
    input_text = re.sub('[^\w\s]', '', input_text)
    predictions = tf_get_predictions(input_text, nn_model)
    crosslingual_tts(predictions, "output.mp3")
    html = predictions_to_html(predictions)
    return html, "output.mp3"


iface = gr.Interface(
    fn=gradio_call,
    inputs=[
        gr.components.Textbox(label="Input", placeholder="Do you sprichst Français?")
    ],
    outputs=[
        gr.components.HTML(), 
        gr.components.Audio(type="filepath")
    ],
    title="Do you sprichst français?",
    description='''
### This app can perform cross-lingual language detection and text-to-speech. \n
### Disclaimer\n
It is built as a proof of concept, that uses machine learning. Specific training data has been created for this project, which might impact the performances with some text categories due to a limited amount of data. This might cause prediction errors when it encounters very short or unfamiliar inputs.\n
Examples that work well:\n
    "How do you say ich arbeite in Zürich in English?"
    "As we say in french: pierre qui roule n'amasse pas mousse!"
    "I have attended a haute couture show last week in Paris."
Examples that might not work:\n
    "Danke!"
    "Une thèse de Diego Fraile et Hugo Perotto."
    ''',
    allow_flagging="never"
)
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:81: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:323: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>